In [1]:
#!/usr/bin/env python
# import analyzer
import importlib
from importlib import reload
import os, sys, glob, warnings, glob
import scipy
import numpy as np
import scipy as sp
import joblib
# from tqdm.notebook import tqdm
from tqdm import tqdm
import copy as cp


import ROOT as root

import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

from pylab import *
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %matplotlib widget

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
Welcome to JupyROOT 6.24/06


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
# Figure configuration are saved in this file include_figure_preset.py
from include_figure_preset import * 
# Set Figure font family, fontsize, ticks, etc.
plt_config(family="san-serif", fontsize_multi=1) # or "serif", or an exact font name


# Redefine a function to save figures with common settings 
fig_prefix = "plots/singletrack_"    # It's good to keep figures in a separate folder. Can also be set to None.
fig_format = "jpg"      # for multiple formats, e.g.: "pdf,png"
SAVE_FIG = False         # Use this flag to turn the figure saving on or off, so that you don't need to modify all notebook to save figure.
# You can then do `savefig(filename_without_extension)` to save your plots with these settings
savefig = Save_fig(fig_prefix=fig_prefix, exts=fig_format, SAVE= SAVE_FIG, dpi=300)

In [4]:
DATA_DIR    = "/project/def-mdiamond/tomren/mathusla/data/fit_study_6layer"

## Universal names, Input file, output filenames, etc

In [ ]:
#--- Get a list of filenames to process ---
energy_list = [0.1, 0.2, 0.5, 1, 3, 10, 30, 100]
name_list = ["muon", "pion", "electron"]
name_list_latex = ["$\mu^-$", "$\pi^+$", "$\e^-$"]
pdgid_list = [13, 211, 11]

INDS_PAR = [2,0,3,6,4,5] # from CMS coord to DET coord. 
PAR_LABELS=["$x_0$ [cm]","$y_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
PAR_LABELS_CMS=["$x_0$ [cm]","$y_0$ [cm]","$z_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
PAR_LABELS_RAW=["x0", "y0", "z0", "t0", "vx", "vy", "vz"]
PAR_PLOT_RANGES=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])

In [12]:
file_list = {}

for name in name_list:
    file_list_temp = []
    for energy in energy_list:
        files=glob.glob(f"{DATA_DIR}/{name}_{energy}_GeV/*/*/stat_seedmod.root",)
        #files=util.Utils.sortByExt(files)
        if len(files)>=1:
            file_list_temp.append(files[0])
        if len(files)>1:
            print(f"More than one file for {name} at {energy} GeV")
    file_list[name] = file_list_temp

# Useful functions

In [16]:
def plot_res_pull(data, figs=None, make_legend=False, plot_gauss1=True, label="", figsize=(10,3.5)):
    axlabels=["$x_0$ [cm]","$y_0$ [cm]", "$t_0$ [ns]", "$v_x$ [cm/ns]", "$v_y$ [cm/ns]", "$v_z$ [cm/ns]"]
    ranges=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])

    mask_recon_success=data["mask_recon_success"]
    recon     =np.array(data["recon"])[mask_recon_success]
    recon_unc =np.array(data["recon_error"])[mask_recon_success]
    truth     =np.array(data["truth"])[mask_recon_success]

    
    if figs is None:
        figs = []
        for i in range(6):
            fig,axs=plt.subplots(1,2,figsize=figsize)
            figs.append(fig)

    for ipar in range(6):
        ipar_cms = INDS_PAR[ipar]
        residual=(recon-truth)[:,ipar_cms]
        pull=util.pull(residual,0,recon_unc[:,ipar_cms])

        # plotrange=[-2*np.std(residual_kf), 2*np.std(residual_kf)]
        plotrange=ranges[ipar]

        # fig,axs=plt.subplots(1,2,figsize=(10,4))
        figure(figs[ipar])
        axs=figs[ipar].axes
        plt.sca(axs[0])
        n,ibins,p = hist(residual,range=plotrange,histtype="step",label=label,bins=100);
        yscale("log")
        ymin, ymax = gca().get_ylim()
        ylim(bottom=1, top = max([max(n)*2,ymax]))        
        xlabel("Reco-truth, "+axlabels[ipar])
        ylabel("[counts/bin]")

        plt.sca(axs[1])
        n,ibins,p = hist(pull,range=[-5,5],histtype="step",label=label,bins=100);

        bincenters=0.5*(ibins[1:]+ibins[:-1])
        y = util.Utils.Gauss(bincenters, max(n),0,1)
        if plot_gauss1:
            plt.plot(bincenters,y,color="r",label=r"Gauss, $\sigma$=1",linestyle=":")
            
        yscale("log")
        ymin, ymax = gca().get_ylim()
        ylim(bottom=1, top = max([max(n)*2,ymax]))
        xlabel(r"$\frac{Reco-truth}{Reco_{unc}}$, "+axlabels[ipar].split(" ")[0]+" [$\sigma$]")
        
        if make_legend:
            plt.legend(loc=(1.01,0),fontsize=11)
    
    return figs

In [16]:
def calc_eff(res, PDG_TRUTH = 13):
    recon     =np.array(res["recon"])
    recon_unc =np.array(res["recon_error"])
    truth     =np.array(res["truth"])
    
    
    mask_recon_success=res["mask_recon_success"]
    mask_recon_able = (res["truth_nlayer"]>=5)& (res["truth_nlayer"]<=9) # layer 2 is the bottom layer, 9 is the top
    
    mask_identified= np.zeros(len(res["recon"]),dtype=bool)
    for i in range(len(mask_identified)):
        n_truth_id = sum(np.array(res["par_km_pdgids"][i])==PDG_TRUTH)
        n_false_id = sum(np.array(res["par_km_pdgids"][i])!=PDG_TRUTH)
        track_purity = n_truth_id/(n_truth_id+n_false_id)
        if n_truth_id>=4 and n_false_id==0:
            mask_identified[i]=True
    
    n_events = len(mask_recon_success)
    n_success = np.sum(mask_recon_success)
    
        
    # Make a fixed range cut for tight and looser track
    diffx = recon[:,2]-truth[:,2]
    diffy = recon[:,0]-truth[:,0]
    diffvx = recon[:,6]-truth[:,6]
    diffvy = recon[:,4]-truth[:,4] 
    mask_TIGHT  = (np.abs(diffx)<5) & (np.abs(diffy)<5) & (np.abs(diffvx)<0.5) & (np.abs(diffvy)<0.5)
    mask_LOOSER = (np.abs(diffx)<10) & (np.abs(diffy)<10) & (np.abs(diffvx)<1) & (np.abs(diffvy)<1)
            
            
    eff_raw=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_recon_success)],[len(mask_recon_success)])))
    eff_abs_tight=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_TIGHT&mask_recon_success)],[len(mask_recon_success)])))
    eff_abs_loose=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_LOOSER&mask_recon_success)],[len(mask_recon_success)])))

    eff_reconstructible = util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_recon_able)],[len(mask_recon_success)])))
    eff_identified=util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_identified&mask_recon_able)],[sum(mask_recon_able)])))
    eff_resolution =util.Utils.flatten1d(list(rt.BayesDivide([sum(mask_LOOSER&mask_identified&mask_recon_able)],[sum(mask_identified&mask_recon_able)])))
    
    if len(eff_resolution)==0:
        eff_resolution = [0,0,0]
    
    return eff_raw, eff_abs_tight, eff_abs_loose, eff_reconstructible,  eff_identified, eff_resolution


def calc_resolution(res):
    recon     =np.array(res["recon"])
    recon_unc =np.array(res["recon_error"])
    truth     =np.array(res["truth"])
    
    
    mask_recon_success=res["mask_recon_success"]
    ranges=np.array([[-15,15],[-15,15],[-4,4],[-2,2], [-2,2], [-6,6]])
    ind = INDS_PAR
    
    sigmas = []
    sigmas_unc = []
    fwhms = []
    for i in range(6):
        diff = recon[:,ind[i]]-truth[:,ind[i]]
        n,ibins= np.histogram(diff,bins=100,range=ranges[i]);
        
        
        bincenters=0.5*(ibins[1:]+ibins[:-1])
        yerr=np.sqrt(n);yerr[yerr==0]=1
        
        popt,pcov = rt.fit_tg(bincenters,n,yerr=yerr,function="gaus")
        perr = np.sqrt(np.diag(pcov))
        
        fw = util.Utils.fwhm(bincenters, n)
        fwhm = fw[1]-fw[0]
        
        sigmas.append(popt[2])
        sigmas_unc.append(perr[2])
        fwhms.append(fwhm)
        
    return sigmas,sigmas_unc,fwhms
    